In [140]:
import os
import json
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### 1. Removing the Cities with No Restaurent Listed

In [141]:
# 1. Loading the JSON
js = open('/kaggle/input/swiggy-restaurants-dataset/data.json','r')
data = json.loads(js.read())
js.close()


# 2. Deleting the Cities with No Restaurants
for city in list(data.keys()):
    if ('restaurants' in data[city].keys()):                
        if (len(data[city]['restaurants']) == 0):
            del data[city]
            
# 3. Deleting the Subregions with No Restaurants
for i in range(100):    
    try:
        for city in list(data.keys()):
            if ('restaurants' not in data[city].keys()):
                dct = data[city]
                for subregion in dct:                          
                    if (subregion != 'link'):                                
                        if (len(dct[subregion]['restaurants']) == 0):
                            del dct[subregion]
                            data[city] = dct
    except:
        pass

### 2. Removing the Restaurants with no Menu

In [142]:
# 1. Removing data for Cities without Subregions
for _ in tqdm(range(2000)):
    try:                                                                                              # Get rid of Dictionary Changing Size Error
        for city in list (data.keys()):                                                               # Going through all the cities
            if ('restaurants' in data[city].keys()) :                                                 # Cities without Subregions
                for rest in data[city]['restaurants' ].keys() :                                       # Going through all the Restaurants
                    if ('menu' in data[city]['restaurants' ][rest].keys()):                            # If Menu is there
                        if ((len(data[city]['restaurants' ][rest]['menu'])) == 0) :                   # Empty Menu
                            del data[city]['restaurants' ][rest]                                      # Deleting the Restaurant Data
                    else:                                                                             # If Menu is not there
                        del datal[city]['restaurants'][rest]                                          # Deleting the Restaurant Data
    


    except:
        pass
    
# 2. Removing data for Cities with subregions
for _ in tqdm(range(2000)):
    try:                                                                                              # Get rid of Dictionary Changing Size Error
        for city in list (data.keys()):                                                               # Going through all the cities
            if ('restaurants' not in data[city].keys()):                                              # Cities with Subregions
                for subregion in data[city].keys():                                                   # Going through all the Subregions
                    for rest in data[city][subregion]['restaurants'].keys():                          # Going through all the restaurants
                        if ('menu' in data[city][subregion]['restaurants'][rest].keys()):             # If Menu is there
                            if (len(data[city][subregion]['restaurants' ][rest]['menu']) == 0):       # If Menu is Empty
                                del data[city][subregion]['restaurants' ][rest]                       # Deleting the Restaurant Data
                        else:                                                                         # If Menu is not there
                            del data[city][subregion]['restaurants' ][rest]                           # Deleting the Restaurant Data

    except:
        pass

### 3. Saving the JSON

In [4]:
# 6. Saving the Preprocessed Dataset
fd = open('swiggy.json','w')
fd.write(json.dumps(data))
fd.close()

### 4. Saving Individual JSON File for Each City

In [144]:
#!mkdir Cities

for city in tqdm(data.keys()):
    
    fd = open('Cities/' + city + '.json','w')
    fd.write(json.dumps(data[city]))
    fd.close()    

### 5. Saving the data in CSV Format
##### 5.1) Gathering the Data in a List

In [84]:
# !mkdir Menu                                                                                  # Create a Dicrectory named as Menu

swiggy = []

rest_name         = ''
rest_rating       = ''
rest_rating_count = ''
rest_cost         = ''
rest_address      = ''
rest_cuisine      = ''
rest_lic          = ''
rest_link         = ''

for city in tqdm(list(data.keys())):
    if ('restaurants' in data[city].keys()):                                                    # Without Subregions          
        for rest in data[city]['restaurants'].keys():                                           # Going through Each Restaurant
            try:
                rest_city          = city
            except:
                rest_city          = np.nan
            try:
                rest_id            = rest
            except:
                rest_id            = np.nan
            try:
                rest_name          = data[city]['restaurants'][rest]['name']
            except:
                rest_name          = np.nan 
            try:
                rest_rating        = data[city]['restaurants'][rest]['rating']
            except:
                rest_rating        = np.nan   
            try:
                rest_rating_count  = data[city]['restaurants'][rest]['rating_count']
            except:
                rest_rating_count  = np.nan
            try:  
                rest_cost          = data[city]['restaurants'][rest]['cost']
            except:
                rest_cost          = np.nan
            try:
                rest_address       = data[city]['restaurants'][rest]['address']
            except:
                rest_address       = np.nan
            try:
                rest_cuisine       = data[city]['restaurants'][rest]['cuisine']
            except:
                rest_cuisine       = np.nan
            try:
                rest_lic           = data[city]['restaurants'][rest]['lic_no']
            except:
                rest_lic           = np.nan
            try:
                rest_link          = data[city]['restaurants'][rest]['link']
            except:
                rest_link          = np.nan
            try:
                rest_menu          = data[city]['restaurants'][rest]['menu']                    # Loading the Menu       
                fd                 = open('Menu/' + rest_id + '.json','w')                      # Opening a JSON
                fd.write(json.dumps(rest_menu))                                                 # Saving Menu in the JSON
                fd.close()                                                                      # Closing the File
                rest_menu          = 'Menu/' + rest_id + '.json'                               # Taking File Path
            except:
                rest_menu          = np.nan
                    
        
            swiggy.append([rest_id,rest_name, rest_city,rest_rating, rest_rating_count, 
                        rest_cost, rest_cuisine, rest_lic, rest_link, rest_address, rest_menu])
            
    else:                                                                                      # With Subregions     
        for subregion in data[city].keys():                                                    # Going through all Subregions    
            for rest in data[city][subregion]['restaurants'].keys():                           # Going through Each Restaurant
                
                try:
                    rest_city          = subregion +','+city
                except:
                    rest_city          = np.nan
                try:
                    rest_id            = rest
                except:
                    rest_id            = np.nan
                try:
                    rest_name          = data[city][subregion]['restaurants'][rest]['name']
                except:
                    rest_name          = np.nan
                try:    
                    rest_rating        = data[city][subregion]['restaurants'][rest]['rating']
                except:
                    rest_rating        = np.nan
                try:    
                    rest_rating_count  = data[city][subregion]['restaurants'][rest]['rating_count']
                except:
                    rest_rating_count  = np.nan
                try:    
                    rest_cost          = data[city][subregion]['restaurants'][rest]['cost']
                except:
                    rest_cost          = np.nan
                try:    
                    rest_address       = data[city][subregion]['restaurants'][rest]['address']
                except:
                    rest_address       = np.nan
                try:    
                    rest_cuisine       = data[city][subregion]['restaurants'][rest]['cuisine']
                except:
                    rest_cuisine       = np.nan
                try:
                    rest_lic           = data[city][subregion]['restaurants'][rest]['lic_no']
                except:
                    rest_lic           = np.nan
                try:
                    rest_link          = data[city][subregion]['restaurants'][rest]['link']
                except:
                    rest_link          = np.nan
                try:
                    rest_menu          = data[city][subregion]['restaurants'][rest]['menu']         # Loading the Menu       
                    fd                 = open('Menu/' + rest_id + '.json','w')                      # Opening a JSON
                    fd.write(json.dumps(rest_menu))                                                 # Saving Menu in the JSON
                    fd.close()                                                                      # Closing the File
                    rest_menu          = 'Menu/' + rest_id + '.json'                                # Taking File Path
                except:
                    rest_menu          = np.nan

                swiggy.append([rest_id,rest_name, rest_city,rest_rating, rest_rating_count, 
                               rest_cost, rest_cuisine, rest_lic, rest_link, rest_address, rest_menu])

##### 5.2) Save the Data in a DataFrame

In [87]:
swiggy = pd.DataFrame(swiggy, columns = ['id','name','city','rating','rating_count','cost','cuisine','lic_no','link','address', 'menu'])
print(len(swiggy))

##### 5.3) Data Preprocessing

In [135]:
swiggy = swiggy.dropna()                           # Droping Rows with Null Values
print("Before : ", len(swiggy))
swiggy = swiggy.drop_duplicates(subset = 'id')    # Dropping Duplicate Restaurants based on their restaurant-id
print("After  : ", len(swiggy))

##### 5.4) Saving the CSV

In [150]:
swiggy.to_csv('swiggy.csv', index = False)

##### 5.4) Downloading the Datasets

In [153]:
shutil.make_archive('Cities', 'zip', 'Cities')       # Creating Zip of Cities Folder
shutil.make_archive('Menu', 'zip', 'Menu')           # Creating Zip of Menu Folder